https://github.com/gbrammer/eazy-py/blob/master/docs/examples/HDFN-demo.ipynb  
https://github.com/gbrammer/eazy-py

In [ ]:
%matplotlib inline
import os
os.environ['EAZYCODE'] = 'usr/local/share/python/eazy-py/eazy-photoz'

import os
import numpy as np
import matplotlib.pyplot as plt

import eazy

# Symlink templates & filters from the eazy-code repository
try:
    print('EAZYCODE = '+os.getenv('EAZYCODE'))
except:
    pass

if not os.path.exists('templates'):
    eazy.symlink_eazy_inputs() 
    

# Module versions
import importlib
import sys
import time
print(time.ctime() + '\n')

print(sys.version + '\n')

for module in ['numpy', 'scipy', 'matplotlib','astropy','eazy', 'prospect']:
    #print(module)
    mod = importlib.import_module(module)
    print('{0:>20} : {1}'.format(module, mod.__version__))

In [ ]:
# quiet numpy/astropy warnings
import warnings
from astropy.utils.exceptions import AstropyWarning

np.seterr(all='ignore')
warnings.simplefilter('ignore', category=AstropyWarning)

EAZY translate file has 2 columns:
1) column label in your catalog  
2) filter line number in EAZY filter list prepended with:  
   "F"  for flux in uJy   
   "EF" for flux uncertainty in uJy
   
or "id" for id number; "z_spec" for spectroscopic redshift

NIRCam filters start on line 362:  
https://github.com/gbrammer/eazy-photoz/blob/cdd4510dbc32d828334f8d0b2e88652894a54653/filters/FILTER.RES.latest.info#L362

You might only have to set this up once.

In [ ]:
# Input catalog csv with flux in uJy
field = 'ceers5'
inroot = field + '_photometry'
catalog_file = os.path.join('../phot', inroot+'.csv')
translate_file = 'eazypy.translate.txt'
output_file = field + '.eazypy'

In [ ]:
# EAZY input parameters
# https://eazy-py.readthedocs.io/en/latest/_modules/eazy/photoz.html

params = {}
params['CATALOG_FILE'] = catalog_file
params['CATALOG_FORMAT'] = 'csv'

params['MAIN_OUTPUT_FILE'] = output_file

# Galactic extinction
params['MW_EBV'] = 0.0103
#params['MW_EBV'] = eazy.utils.get_irsa_dust(np.nanmedian(zf['ra']), np.nanmedian(zf['dec']))

params['CAT_HAS_EXTCORR'] = True

params['Z_STEP'] = 0.01
params['Z_MIN'] = 0.01
params['Z_MAX'] = 16.

params['PRIOR_ABZP'] = 25 
params['PRIOR_FILTER'] = 'f200w_flux' # K-band ~ F200W  # previously set to 28 255
params['PRIOR_FILE'] = 'templates/prior_K_TAO.dat'

params['TEMPLATES_FILE'] = 'templates/fsps_full/tweak_fsps_QSF_12_v3.param'
#params['TEMPLATES_FILE'] = 'templates/spline_templates_v2/tweak_spline.param'

# FSPS + bluer templates added by Rebecca Larson 2022:
#params['TEMPLATES_FILE'] = 'templates/Larson22/tweak_fsps_QSF_12_v3_newtemplates_Lya_Reduced.param'  # general purpose
#params['TEMPLATES_FILE'] = 'templates/Larson22/tweak_fsps_QSF_12_v3_newtemplates.param'  # z > 8
# /Users/dcoe/miniconda3/envs/erophot/lib/python3.10/site-packages/eazy/data/templates
# templates/Larson22/tweak_fsps_QSF_12_v3_newtemplates_Lya_Reduced.param.fits 

params['FIX_ZSPEC'] = False

params['IGM_SCALE_TAU'] = 1.0

params['SYS_ERR'] = 0.03

In [ ]:
# Initialize EAZY object
self = eazy.photoz.PhotoZ(param_file=None, translate_file=translate_file, zeropoint_file=None, 
                          params=params, load_prior=True, load_products=False)

In [ ]:
# Run EAZY SED fitting (takes a minute or two for thousands of objects)
self.fit_catalog(self.idx, n_proc=8)

In [ ]:
# Derived parameters (z params, RF colors, masses, SFR, etc.)
# (takes another minute or two for thousands of objects)
warnings.simplefilter('ignore', category=RuntimeWarning)
zout, hdu = self.standard_output(simple=False, 
                                 rf_pad_width=0.5, rf_max_err=2, 
                                 prior=True, beta_prior=True, 
                                 absmag_filters=[], 
                                 extra_rf_filters=[])

# 'zout' also saved to [MAIN_OUTPUT_FILE].zout.fits

In [ ]:
# https://eazy-py.readthedocs.io/en/latest/eazy/zout_columns.html
zout

In [ ]:
# Saves a massive ~500MB file with everything
# (I'd rather rerun EAZY than save such a large file)
if 0:
    try:
        import h5py
    except:
        ! pip install h5py

    import eazy.hdf5
    #eazy.hdf5.write_hdf5(self, h5file=self.param['MAIN_OUTPUT_FILE'] + '.h5')

In [ ]:
if not os.path.exists('plots'):
    os.makedirs('plots')

In [ ]:
# https://eazy-py.readthedocs.io/en/latest/_modules/eazy/photoz.html
id = 754
fig, data = self.show_fit(id, xlim=[0.5, 5], show_fnu=True, maglim=(31,25), 
                          show_components=True, logpz=True, zr=[0,12], show_prior=True)
fig.savefig('plots/eazy_%d.png' % id, dpi=300)

In [ ]:
# https://eazy-py.readthedocs.io/en/latest/_modules/eazy/photoz.html
z8ids = [754, 918, 1166, 1545, 1725, 1859, 2558, 2737, 3944, 4045, 4219, 4688]
for id in z8ids:
    fig, data = self.show_fit(id, xlim=[0.5, 5], show_fnu=True, maglim=(31,25), 
                              show_components=True, logpz=True, zr=[0,12], show_prior=True)
    fig.savefig('plots/eazy_%d.png' % id, dpi=300)

In [ ]:
# Show best SED fit to the input spec-z
z8ids = [754, 918, 1166, 1545, 1725, 1859, 2558, 2737, 3944, 4045, 4219, 4688]
zspecs = [8.291995, 8.948092, 8.284709, 8.302413, 8.290659, 8.246575, 8.295558, 8.283571, 8.289624, 8.286771, 9.497089, 8.932844]
for i, id in enumerate(z8ids):
    zspec = zspecs[i]
    fig, data = self.show_fit(id, xlim=[0.5, 5], show_fnu=True, maglim=(31,25), 
                              zshow=zspec,  # show best SED fit to the input spec-z
                              show_components=True, logpz=True, zr=[0,12], show_prior=False)
    fig.savefig('plots/zspec/eazy_zspec_%d.png' % id, dpi=300)

# Combined output file: photometry + photo-z's

In [ ]:
import astropy

In [ ]:
#catalog = QTable.read(catalog_file)
catalog = astropy.io.ascii.read(catalog_file)

In [ ]:
for label in zout.columns[3:]:
    #print(label)
    catalog[label] = zout[label]

In [ ]:
outfile = field + '_phot-eazy.ecsv'
catalog.write(outfile, overwrite=True)

In [ ]:
# Remove last 5 columns that give percentiles
# (could instead reformat into flat catalog if desired)
for label in list(catalog.columns):
    if label.endswith('_p'):
        catalog.remove_column(label)

outfile = field + '_phot-eazy.cat'
catalog.write(outfile, format='ascii.fixed_width_two_line', delimiter=' ', overwrite=True)

In [ ]:
catalog